# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [41]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)

# Receiving error message that states that the homeowner columns 
# (mortgage or own) cannot be converted into a float for the date testing. This column is subsequently being dropped. 

df = df.drop(columns = 'homeowner')

# Drop the null columns where all values are null
df = df.dropna(axis='columns')

# Drop the null rows
df = df.dropna()


df.head()



loan_size  interest_rate  borrower_income  debt_to_income  num_of_accounts  \
0    10700.0          7.672            52800        0.431818                5   
1     8400.0          6.692            43600        0.311927                3   
2     9000.0          6.963            46100        0.349241                3   
3    10700.0          7.664            52700        0.430740                5   
4    10800.0          7.698            53000        0.433962                5   

   derogatory_marks  total_debt loan_status  
0                 1       22800    low_risk  
1                 0       13600    low_risk  
2                 0       16100    low_risk  
3                 1       22700    low_risk  
4                 1       23000    low_risk

# Split the Data into Training and Testing

In [4]:
# Create our features
X = df.drop(columns = 'loan_status')

# Create our target
y = df['loan_status']

In [5]:
X.describe()

loan_size  interest_rate  borrower_income  debt_to_income  \
count  77536.000000   77536.000000     77536.000000    77536.000000   
mean    9805.562577       7.292333     49221.949804        0.377318   
std     2093.223153       0.889495      8371.635077        0.081519   
min     5000.000000       5.250000     30000.000000        0.000000   
25%     8700.000000       6.825000     44800.000000        0.330357   
50%     9500.000000       7.172000     48100.000000        0.376299   
75%    10400.000000       7.528000     51400.000000        0.416342   
max    23800.000000      13.235000    105200.000000        0.714829   

       num_of_accounts  derogatory_marks    total_debt  
count     77536.000000      77536.000000  77536.000000  
mean          3.826610          0.392308  19221.949804  
std           1.904426          0.582086   8371.635077  
min           0.000000          0.000000      0.000000  
25%           3.000000          0.000000  14800.000000  
50%           4.000000          0.000000  18100.000000  
75%           4.000000          1.000000  21400.000000  
max          16.000000          3.000000  75200.000000

In [6]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [7]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [8]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scaler = StandardScaler()


In [9]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)


In [10]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [12]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]])

In [14]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [15]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler
model = RandomOverSampler(random_state=1)
X_resampled, y_resampled = model.fit_resample(X_train, y_train)
print(model)


# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

RandomOverSampler(random_state=1)


Counter({'low_risk': 56277, 'high_risk': 56277})

In [16]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression()
model.fit(X_resampled, y_resampled)

LogisticRegression()

In [17]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,
                        "Actual": y_test}).reset_index(drop=True)
balanced_accuracy_score(y_test, predictions)


0.9948279972279972

In [18]:
# Display the confusion matrix
# YOUR CODE HERE
predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,"Actual": y_test}).reset_index(drop=True)
confusion_matrix(y_test, predictions)

array([[  622,     3],
       [  104, 18655]])

In [19]:
# Print the imbalanced classification report
# YOUR CODE HERE
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

   high_risk       0.86      1.00      0.92       625
    low_risk       1.00      0.99      1.00     18759

    accuracy                           0.99     19384
   macro avg       0.93      0.99      0.96     19384
weighted avg       1.00      0.99      0.99     19384



### SMOTE Oversampling

In [20]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE().fit_resample(
    X_train, y_train
)

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [21]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression()
model.fit(X_resampled, y_resampled)

LogisticRegression()

In [22]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_prediction = model.predict(X_test)
balanced_accuracy_score(y_test, y_prediction)

0.9947746894823818

In [23]:
# Display the confusion matrix
# YOUR CODE HERE
pd.DataFrame(
    confusion_matrix(y_test, y_pred), 
    index=["High Risk", "Low Risk"], 
    columns=["Positive", "Negative"]
)

Positive  Negative
High Risk       571        54
Low Risk         93     18666

In [24]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [25]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE

from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids()
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [26]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression()
model.fit(X_resampled, y_resampled)

LogisticRegression()

In [27]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9860813049736127

In [28]:
# Display the confusion matrix
# YOUR CODE HERE
pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=["High Risk", "Low Risk"], 
    columns=["Positive", "Negative"]
)

Positive  Negative
High Risk       611        14
Low Risk        102     18657

In [29]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.98      0.99      0.91      0.99      0.97       625
   low_risk       1.00      0.99      0.98      1.00      0.99      0.97     18759

avg / total       0.99      0.99      0.98      0.99      0.99      0.97     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [36]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN()
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)


# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_resampled)

Counter({'high_risk': 55275, 'low_risk': 55883})

In [37]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression()
model.fit(X_resampled, y_resampled)


LogisticRegression()

In [38]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.994748035609574

In [39]:
# Display the confusion matrix
# YOUR CODE HERE
pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=["High Risk", "Low Risk"],
    columns=["Positive", "Negative"]
)

Positive  Negative
High Risk       622         3
Low Risk        107     18652

In [40]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

       Naive Random Oversampling

2. Which model had the best recall score?
        All the same

3. Which model had the best geometric mean score?
        Over & Under Sampling and Undersampling 
